In [1]:
import os
import pandas as pd
import glob
import menpo.io as mio
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


In [2]:
# path to the model weights file.
weights_path = './vgg16_weights.h5'
top_model_weights_path = './fc_model.h5'

data_file1 = 'idToLabel.tsv'
data_dir1 = './croppedTwitter'
data_file2 = 'gold.tsv'
data_dir2 = './cropped'
variable = 'ow' # relevant column of data_file

# dimensions of our images.
img_width, img_height = 150, 150
channels = 3

In [3]:
# returns (X_train, y_train), (X_test, y_test)
def randomSplit(X, y, propTest):
    assert X.shape[0] == y.shape[0]
    p = np.random.permutation(len(y))
    sX, sy = X[p], y[p]
    cutoff = round(propTest * len(y))
    return (sX[cutoff:], sy[cutoff:]), (sX[0:cutoff], sy[0:cutoff])

def randomOrder(X, y):
    p = np.random.permutation(len(y))
    return X[p], y[p]

# based on
# http://www.socouldanyone.com/2013/03/converting-grayscale-to-rgb-with-numpy.html
def to_rgb(im):
    if im.n_channels == 1:
        return np.vstack([im.pixels.astype(np.uint8)] * 3)
    else:
        return im.pixels

def subsample(df, col, props = None):
    labels = df[col].get_values()
    counts = dict()
    for l in labels:
        counts[l] = counts.get(l, 0) + 1
    countsOfProps = dict()
    for l in set(labels):
        countsOfProps[l] = counts.get(l, 0) * props.get(l, 0)
    leastKey = min(countsOfProps, key=countsOfProps.get)
    leastVal = round(countsOfProps[leastKey])
    thresholds = dict()
    for l in set(labels):
        if props == None or not leastKey in props: # assume equal parts
            thresholds[l] = leastVal
        else:
            labelToLabel = props.get(l, 0) / props[leastKey]
            thresholds[l] = round(labelToLabel * leastVal)
    df = df.reindex(np.random.permutation(df.index))
    enough = dict()
    ret = pd.DataFrame(index=np.arange(0, leastVal / props[leastKey]), columns=df.columns)
    i = 0
    for index, r in df.iterrows():
        if(enough.get(r[col], 0) <= thresholds[r[col]]):
            ret.loc[i] = r
            enough[r[col]] = enough.get(r[col], 0) + 1
            i += 1
    return ret

In [22]:
props = { 1: 0.5, 0: 0.5 }

annos1 = pd.DataFrame.from_csv(data_file1, header=0, index_col=None, sep="\t")
annos1['id'] = [str(int(x)) for x in annos1['id']]
annos1 = subsample(annos1, variable, props)

annos2 = pd.DataFrame.from_csv(data_file2, header=0, index_col=None, sep="\t")
annos2 = subsample(annos2, variable, props)

ids = list(set(annos1['id'].get_values())) + list(set(annos2['subject_id'].get_values()))
ids = np.random.permutation(ids)
boundary = round(0.75 * len(ids))
train_ids = set(ids[0:boundary])
test_ids = set(ids[boundary:])

train_images = []
train_labels = []
test_images = []
test_labels = []
for fp in glob.glob(data_dir1 + '/*.jpg'):
    fnWithExtension = os.path.basename(fp)
    fn = os.path.splitext(fnWithExtension)[0]
    image = mio.import_image(fp)
    assert(image.shape == (img_height, img_width))
    if fn in train_ids:
        train_images.append(image)
        train_labels.append(annos1.loc[annos1['id'] == fn][variable].sum())
    elif fn in test_ids:
        test_images.append(image)
        test_labels.append(annos1.loc[annos1['id'] == fn][variable].sum())

for fp in glob.glob(data_dir2 + '/*.jpg'):
    fn = os.path.basename(fp)
    image = mio.import_image(fp)
    assert(image.shape == (img_height, img_width))
    id = str(annos2.loc[annos2['file_name'] == fn]['subject_id'].sum())
    if id in train_ids:
        train_images.append(image)
        train_labels.append(annos2.loc[annos2['file_name'] == fn][variable].sum())
    elif id in test_ids:
        test_images.append(image)
        test_labels.append(annos2.loc[annos2['file_name'] == fn][variable].sum())


In [24]:
nb_classes = 2

X_train = np.stack([to_rgb(img) for img in train_images])
X_test = np.stack([to_rgb(img) for img in test_images])
if K.image_dim_ordering() == 'tf':
    X_train = X_train.reshape(X_train.shape[0], img_height, img_width, channels)
    X_test = X_test.reshape(X_test.shape[0], img_height, img_width, channels)
    input_shape = input_shape = (img_height, img_width, channels)
else:
    input_shape = (channels, img_height, img_width)
    
y_train = np.array(train_labels)
y_test = np.array(test_labels)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to n-ary class matrices
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

print('y_train shape:', y_train.shape)

X_train, y_train = randomOrder(X_train, y_train)
X_test, y_test = randomOrder(X_test, y_test)

X_train shape: (2985, 150, 150, 3)
2985 train samples
999 test samples
y_train shape: (2985, 2)


In [25]:
# build the VGG16 network
model_vgg = Sequential()
model_vgg.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

In [26]:
# load the weights of the VGG16 networks
# (trained on ImageNet, won the ILSVRC competition in 2014)
# note: when there is a complete match between your model definition
# and your weight savefile, you can simply call model.load_weights(filename)
assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
import h5py
f = h5py.File(weights_path)
for k in range(f.attrs['nb_layers']):
    if k >= len(model_vgg.layers) - 1:
        # we don't look at the last two layers in the savefile (fully-connected and activation)
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    layer = model_vgg.layers[k]

    if layer.__class__.__name__ in ['Convolution1D', 'Convolution2D', 'Convolution3D', 'AtrousConvolution2D']:
        weights[0] = np.transpose(weights[0], (2, 3, 1, 0))

    layer.set_weights(weights)

f.close()

In [29]:
batch_size = 32
nb_train_samples = y_train.shape[0]
nb_validation_samples = y_test.shape[0]

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    zoom_range=0.2,
    horizontal_flip=True
)
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
)
validation_generator = validation_datagen.flow(X_test, y_test, batch_size=batch_size)

bottleneck_features_train = model_vgg.predict_generator(train_generator, nb_train_samples)
np.save(open('bottleneck_features_train.npy', 'wb'), bottleneck_features_train)

bottleneck_features_validation = model_vgg.predict_generator(validation_generator, nb_validation_samples)
np.save(open('bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

In [30]:
train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))

train_labels = y_train
# train_labels = np.zeros(y_train.shape[0])
# for i in range(y_train.shape[0]):
#     if y_train[i][1] == 1:
#         train_labels[i] = 1
# train_labels = train_labels.reshape((-1,1))

validation_labels = y_test
# validation_labels = np.zeros(y_test.shape[0])
# for i in range(y_test.shape[0]):
#     if y_test[i][1] == 1:
#         validation_labels[i] = 1
# validation_labels = validation_labels.reshape((-1,1))

# train_generator = train_datagen.flow(train_data, train_labels, batch_size=batch_size)
# validation_generator = validation_datagen.flow(validation_data, validation_labels, batch_size=batch_size)

In [31]:
model_top = Sequential()
model_top.add(Flatten(input_shape=train_data.shape[1:], name='flatten1'))
model_top.add(Dense(256, activation='relu', name='dense1'),)
model_top.add(Dropout(0.5, name='dropout1'))
model_top.add(Dense(2, activation='softmax', name='dense2'))

model_top.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
nb_epoch = 20

model_top.fit(train_data,
              train_labels,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_data=(validation_data, validation_labels))
model_top.save_weights(top_model_weights_path)

Train on 2985 samples, validate on 999 samples
Epoch 1/20
2985/2985 [==============================] - 4s - loss: 0.6932 - acc: 0.4992 - val_loss: 0.6932 - val_acc: 0.4905  64/2985 [..............................] - ETA: 3s - loss: 0.6931 - acc: 0.5469

In [34]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

# build the VGG16 network
model_vgg = Sequential()
model_vgg.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

In [35]:
# load the weights of the VGG16 networks
# (trained on ImageNet, won the ILSVRC competition in 2014)
# note: when there is a complete match between your model definition
# and your weight savefile, you can simply call model.load_weights(filename)
assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
import h5py
f = h5py.File(weights_path)
for k in range(f.attrs['nb_layers']):
    if k >= len(model_vgg.layers) - 1:
        # we don't look at the last two layers in the savefile (fully-connected and activation)
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    layer = model_vgg.layers[k]

    if layer.__class__.__name__ in ['Convolution1D', 'Convolution2D', 'Convolution3D', 'AtrousConvolution2D']:
        weights[0] = np.transpose(weights[0], (2, 3, 1, 0))

    layer.set_weights(weights)

f.close()

In [36]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model_vgg.output_shape[1:]))
top_model.add(Dense(256, activation='relu', name='dense1'))
top_model.add(Dropout(0.5, name='dropout'))
top_model.add(Dense(2, activation='softmax', name='dense2'))

top_model.load_weights(top_model_weights_path)

model_vgg.add(top_model)

In [37]:
for layer in model_vgg.layers[:25]:
    layer.trainable = False

In [38]:
from keras import optimizers

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model_vgg.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [39]:
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
validation_generator = test_datagen.flow(X_test, y_test, batch_size=batch_size)

In [40]:
nb_epoch = 2

# fine-tune the model
model_vgg.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

Epoch 1/2
2985/2985 [==============================] - 313s - loss: 0.6931 - acc: 0.5025 - val_loss: 0.6932 - val_acc: 0.4905

In [41]:
model_vgg.save_weights('finetuned_vgg.h5')

In [42]:
model_vgg.load_weights('finetuned_vgg.h5')

In [43]:
model_vgg.evaluate_generator(validation_generator, nb_validation_samples)

[0.69329111377994812, 0.48048048065947341]